<a href="https://colab.research.google.com/github/VincentBEDU/UrbanSim/blob/main/OD_scenarioB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part 2  ScenarioB


# IV Scenarios B

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import seaborn as sns
import folium
import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy.stats
import numpy as np
from math import sqrt

In [4]:
# read London flow data
path="/content/drive/MyDrive/Dataset/london_flows.csv"
londflow = pd.read_csv(path)

In [5]:
londflow.head()

,station_origin,station_destination,flows,population,jobs,distance
0,Abbey Road,Bank and Monument,0,599,78549,8131.525097
1,Abbey Road,Beckton,1,599,442,8510.121774
2,Abbey Road,Blackwall,3,599,665,3775.448872
3,Abbey Road,Canary Wharf,1,599,58772,5086.514220
4,Abbey Road,Canning Town,37,599,15428,2228.923167


In [6]:
# Assuming londflow is your DataFrame and 'distance' is the column name
# Remove rows where the 'distance' column has a value of 0
londflow1 = londflow[londflow['distance'] != 0]

# Now, londflow contains only the rows where distance is not zero

In [7]:

# Filter rows where 'station_destination' is 'Canary Wharf'
canary_wharf_data = londflow1[londflow1['station_destination'] == 'Canary Wharf']

# Display the head of the filtered DataFrame along with the 'jobs' column
print(canary_wharf_data[['station_destination', 'jobs']].head())


    station_destination   jobs
3          Canary Wharf  58772
126        Canary Wharf  58772
348        Canary Wharf  58772
595        Canary Wharf  58772
817        Canary Wharf  58772


In [9]:
# Filter rows where 'station_destination' is 'Canary Wharf'
canary_wharf_data = londflow1[londflow1['station_destination'] == 'Canary Wharf']

# Display the head of the filtered DataFrame along with the 'jobs' column
print(canary_wharf_data[['station_destination', 'jobs']].head())

    station_destination   jobs
3          Canary Wharf  58772
126        Canary Wharf  58772
348        Canary Wharf  58772
595        Canary Wharf  58772
817        Canary Wharf  58772


In [10]:
londflow1.head()

,station_origin,station_destination,flows,population,jobs,distance
0,Abbey Road,Bank and Monument,0,599,78549,8131.525097
1,Abbey Road,Beckton,1,599,442,8510.121774
2,Abbey Road,Blackwall,3,599,665,3775.448872
3,Abbey Road,Canary Wharf,1,599,58772,5086.514220
4,Abbey Road,Canning Town,37,599,15428,2228.923167


In [11]:
#set up the metric calculations
def CalcRSqaured(observed, estimated):
    """Calculate the r^2 from a series of observed and estimated target values
    inputs:
    Observed: Series of actual observed values
    estimated: Series of predicted values"""

    r, p = scipy.stats.pearsonr(observed, estimated)
    R2 = r **2

    return R2

def CalcRMSE(observed, estimated):
    """Calculate Root Mean Square Error between a series of observed and estimated values
    inputs:
    Observed: Series of actual observed values
    estimated: Series of predicted values"""

    res = (observed -estimated)**2
    RMSE = round(sqrt(res.mean()), 3)

    return RMSE

## Production Constrained Model

In [12]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

#take the variables and produce logarithms of them
x_variables = ["population", "jobs", "distance"]
log_x_vars = []
for x in x_variables:
    londflow1[f"log_{x}"] = np.log(londflow1[x])
    log_x_vars.append(f"log_{x}")

/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
<ipython-input-12-9e7bb6219314>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  londflow1[f"log_{x}"] = np.log(londflow1[x])
/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
<ipython-input-12-9e7bb6219314>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexi

In [13]:
londflow1.head()

,station_origin,station_destination,flows,population,jobs,distance,log_population,log_jobs,log_distance
0,Abbey Road,Bank and Monument,0,599,78549,8131.525097,6.395262,11.271478,9.003504
1,Abbey Road,Beckton,1,599,442,8510.121774,6.395262,6.091310,9.049012
2,Abbey Road,Blackwall,3,599,665,3775.448872,6.395262,6.499787,8.236275
3,Abbey Road,Canary Wharf,1,599,58772,5086.514220,6.395262,10.981421,8.534348
4,Abbey Road,Canning Town,37,599,15428,2228.923167,6.395262,9.643939,7.709274


In [14]:
# Check for missing values
print(londflow1.isnull().sum())

# Check data types
print(londflow1.dtypes)

station_origin         0
station_destination    0
flows                  0
population             0
jobs                   0
distance               0
log_population         0
log_jobs               0
log_distance           0
dtype: int64
station_origin          object
station_destination     object
flows                    int64
population               int64
jobs                     int64
distance               float64
log_population         float64
log_jobs               float64
log_distance           float64
dtype: object


In [15]:
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf

try:
    # Filter rows where 'flows' is not zero
    filtered_londflow1 = londflow1[londflow1['flows'] != 0]

    # Create the formula (the "-1" indicates no intercept in the regression model).
    formula = 'flows ~ station_origin + log_jobs + log_distance - 1'

    # Run a production-constrained simulation with increased max iterations
    prodSim = smf.glm(formula=formula, data=filtered_londflow1, family=sm.families.Poisson()).fit(maxiter=1000)
    print(prodSim.summary())
except LinAlgError as e:
    print("Error:", e)


                 Generalized Linear Model Regression Results                  
Dep. Variable:                  flows   No. Observations:                43945
Model:                            GLM   Df Residuals:                    43545
Model Family:                 Poisson   Df Model:                          399
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -9.1409e+05
Date:                Fri, 26 Apr 2024   Deviance:                   1.6560e+06
Time:                        19:32:06   Pearson chi2:                 2.41e+06
No. Iterations:                     7   Pseudo R-squ. (CS):              1.000
Covariance Type:            nonrobust                                         
                                                  coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------

So, what do we have?

Well, there are the elements of the model output that should be familiar from the unconstrained model:

The $\gamma$ parameter related to the destination attractiveness: 0.7302

The $\beta$ distance decay parameter: 0.8152. Recall the negative sign in the equation.

We can see from the standard outputs from the model that all of the explanatory variables are statistically significant (P>|z| < 0.01) and the z-scores indicate that the destination salary is having the most influence on the model, with distance following closely behind. And then we have a series of paramaters which are the vector of $\alpha_i$ values associated with our origin constraints.

## Model Estimates
Now at this point you will want to know what effect the constraints have had on the estimates produced by the model, so let's plug the parameters back into Equation 4 and look...


In [16]:
#create some Oi and Dj columns in the dataframe and store row and column totals in them:
#to create O_i, take cdatasub ...then... group by origcodenew ...then... summarise by calculating the sum of Total
O_i = pd.DataFrame(filtered_londflow1.groupby(["station_origin"])["flows"].agg(np.sum))
O_i.rename(columns={"flows":"O_i"}, inplace = True)
filtered_londflow1 = filtered_londflow1.merge(O_i, on = "station_origin", how = "left" )

D_j = pd.DataFrame(filtered_londflow1.groupby(["station_destination"])["flows"].agg(np.sum))
D_j.rename(columns={"flows":"D_j"}, inplace = True)
filtered_londflow1 = filtered_londflow1.merge(D_j, on = "station_destination", how = "left" )

In [17]:
filtered_londflow1.head()

,station_origin,station_destination,flows,population,jobs,distance,log_population,log_jobs,log_distance,O_i,D_j
0,Abbey Road,Beckton,1,599,442,8510.121774,6.395262,6.091310,9.049012,599,442
1,Abbey Road,Blackwall,3,599,665,3775.448872,6.395262,6.499787,8.236275,599,665
2,Abbey Road,Canary Wharf,1,599,58772,5086.514220,6.395262,10.981421,8.534348,599,58772
3,Abbey Road,Canning Town,37,599,15428,2228.923167,6.395262,9.643939,7.709274,599,15428
4,Abbey Road,Crossharbour,1,599,1208,6686.475560,6.395262,7.096721,8.807842,599,1208


Now, fishing out the coeff. from the prodsim glm object

In [18]:
#We can do this by pulling out the parameter values
coefs = pd.DataFrame(prodSim.params)
coefs.reset_index(inplace=True)
coefs.rename(columns = {0:"alpha_i", "index":"coef"}, inplace = True)
to_repl = ["(station_origin)", "\[", "\]"]
for x in to_repl:
    coefs["coef"] = coefs["coef"].str.replace(x, "",regex=True)
#then once you have done this you can join them back into the dataframes
filtered_londflow1 = filtered_londflow1.merge(coefs, left_on="station_origin", right_on="coef", how = "left")
filtered_londflow1.drop(columns = ["coef"], inplace = True)
#check this has worked
filtered_londflow1.head()

,station_origin,station_destination,flows,population,jobs,distance,log_population,log_jobs,log_distance,O_i,D_j,alpha_i
0,Abbey Road,Beckton,1,599,442,8510.121774,6.395262,6.091310,9.049012,599,442,3.270351
1,Abbey Road,Blackwall,3,599,665,3775.448872,6.395262,6.499787,8.236275,599,665,3.270351
2,Abbey Road,Canary Wharf,1,599,58772,5086.514220,6.395262,10.981421,8.534348,599,58772,3.270351
3,Abbey Road,Canning Town,37,599,15428,2228.923167,6.395262,9.643939,7.709274,599,15428,3.270351
4,Abbey Road,Crossharbour,1,599,1208,6686.475560,6.395262,7.096721,8.807842,599,1208,3.270351


OK, now we can save our parameter values into some variables...

In [21]:
# Creating a New variables for beta

alpha_i = prodSim.params[0:398]
gamma = prodSim.params[398]
beta = -prodSim.params[399]
#beta1 = 0.70           # not forgetting the negative in the formular
#beta2 = 0.78

In [22]:
alpha_i

station_origin[Abbey Road]          3.270351
station_origin[Acton Central]       5.008886
station_origin[Acton Town]          4.397394
station_origin[Aldgate]             3.361125
station_origin[Aldgate East]        3.408728
                                      ...   
station_origin[Wood Street]         5.672160
station_origin[Woodford]            4.955425
station_origin[Woodgrange Park]     5.320215
station_origin[Woodside Park]       4.496709
station_origin[Woolwich Arsenal]    6.701868
Length: 398, dtype: float64

In [23]:
gamma

0.7301699265793882

In [24]:
beta

0.8151874614788027

Ready to generate some estimates

In [25]:
filtered_londflow1["prodsimest1"] = np.exp(filtered_londflow1["alpha_i"]+gamma*filtered_londflow1["log_jobs"]
                                 - beta*filtered_londflow1["log_distance"])
#or you could do it the easy way like we did last week with the fitted column (See previous practical)
filtered_londflow1.head(10)

,station_origin,station_destination,flows,population,jobs,distance,log_population,log_jobs,log_distance,O_i,D_j,alpha_i,prodsimest1
0,Abbey Road,Beckton,1,599,442,8510.121774,6.395262,6.091310,9.049012,599,442,3.270351,1.406918
1,Abbey Road,Blackwall,3,599,665,3775.448872,6.395262,6.499787,8.236275,599,665,3.270351,3.677350
2,Abbey Road,Canary Wharf,1,599,58772,5086.514220,6.395262,10.981421,8.534348,599,58772,3.270351,76.062924
3,Abbey Road,Canning Town,37,599,15428,2228.923167,6.395262,9.643939,7.709274,599,15428,3.270351,56.123931
4,Abbey Road,Crossharbour,1,599,1208,6686.475560,6.395262,7.096721,8.807842,599,1208,3.270351,3.568414
5,Abbey Road,Cutty Sark,2,599,1748,8503.898909,6.395262,7.466228,9.048280,599,1748,3.270351,3.841726
6,Abbey Road,Cyprus,7,599,850,6532.099618,6.395262,6.745236,8.784484,599,850,3.270351,2.813752
7,Abbey Road,Devons Road,1,599,611,3958.324171,6.395262,6.415097,8.283576,599,611,3.270351,3.326081
8,Abbey Road,East India,2,599,1522,3384.141666,6.395262,7.327781,8.126856,599,1522,3.270351,7.359304
9,Abbey Road,Island Gardens,2,599,691,7706.296370,6.395262,6.538140,8.949793,599,691,3.270351,2.113923


## Assessing the model output

So what do the outputs from our Production Constrained Model look like? How has the goodness-of-fit improved and how can we start to use this a bit like a retail model and assess the likley impacts of changing detsination attractiveness etc.?

### The flow matrics

In [26]:
#first round the estimates
#filtered_londflow1["prodsimest1"] = round(filtered_londflow1["prodsimest1"],0)
#now we can create a pivot tabel to turn the paired list into a matrix, and compute the margins as well
londflowmat3 = filtered_londflow1.pivot_table(values ="prodsimest1", index="station_origin", columns = "station_destination",
                            aggfunc=np.sum, margins=True)

londflowmat3.fillna(0, inplace=True)

londflowmat3

station_destination,Abbey Road,Acton Central,Acton Town,Aldgate,Aldgate East,All Saints,Alperton,Amersham,Anerley,Angel,...,Wimbledon,Wimbledon Park,Wood Green,Wood Lane,Wood Street,Woodford,Woodgrange Park,Woodside Park,Woolwich Arsenal,All
station_origin,,,,,,,,,,,,,,,,,,,,,
Abbey Road,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.080449,599.0
Acton Central,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1224.0
Acton Town,0.000000,0.000000,0.000000,20.113147,20.161026,0.000000,9.812411,0.000000,0.000000,21.971277,...,18.345025,3.593363,6.111567,13.966017,0.000000,0.000000,0.000000,0.000000,0.000000,3745.0
Aldgate,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,26.397174,...,0.000000,0.000000,3.537277,2.867179,0.000000,0.000000,0.000000,0.000000,0.000000,2886.0
Aldgate East,0.000000,0.000000,2.942747,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,26.217503,...,5.759102,0.000000,0.000000,2.959688,0.000000,1.484809,0.000000,1.308642,0.000000,3172.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Woodford,0.000000,0.000000,8.426657,37.212112,40.784321,0.000000,0.000000,0.000000,0.000000,34.473399,...,17.200010,0.000000,10.962554,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4868.0
Woodgrange Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,530.0
Woodside Park,0.000000,0.000000,6.211346,21.790171,21.846134,0.000000,0.000000,0.000000,0.000000,26.202682,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3093.0


In [27]:
def CalcRSquared(observed, estimated):
    """Calculate the r^2 from a series of observed and estimated target values
    inputs:
    Observed: Series of actual observed values
    estimated: Series of predicted values"""

    r, p = scipy.stats.pearsonr(observed, estimated)
    R2 = r **2

    return R2

def CalcRMSE(observed, estimated):
    """Calculate Root Mean Square Error between a series of observed and estimated values
    inputs:
    Observed: Series of actual observed values
    estimated: Series of predicted values"""

    res = (observed -estimated)**2
    RMSE = round(sqrt(res.mean()), 3)

    return RMSE

In [28]:
CalcRSquared(filtered_londflow1["flows"], filtered_londflow1["prodsimest1"])

0.3937304850830748

In [29]:
CalcRMSE(filtered_londflow1["flows"], filtered_londflow1["prodsimest1"])

120.146

Clearly by constraining our model estimates to known origin totals, the fit of the model has improved quite considerably - from around 0.3114 in the unconstrained model to around 0.3937 in this model. The RMSE has also dropped quite noticeably (from 128.178 to 120.146)

# Scenario B

To reflect the significant increase in the cost of transport (distance) the actual beta has to be reduced. The original or actual beta = -0.8151874614788027. Therefore, beta = -0.65 and beta = -0.75 will be adopted.

In [34]:
# Creating a New variables for beta

alpha_i = prodSim.params[0:398]
gamma = prodSim.params[398]
#beta = -prodSim.params[399]
beta1 = 0.72           # not forgetting the negative in the formular
beta2 = 0.78

In [35]:
alpha_i

station_origin[Abbey Road]          3.270351
station_origin[Acton Central]       5.008886
station_origin[Acton Town]          4.397394
station_origin[Aldgate]             3.361125
station_origin[Aldgate East]        3.408728
                                      ...   
station_origin[Wood Street]         5.672160
station_origin[Woodford]            4.955425
station_origin[Woodgrange Park]     5.320215
station_origin[Woodside Park]       4.496709
station_origin[Woolwich Arsenal]    6.701868
Length: 398, dtype: float64

In [36]:
gamma

0.7301699265793882

In [37]:
beta1

0.72

## Ready to generate the new  estimate with beta = 0.72

In [38]:
filtered_londflow1["prodsimest2"] = np.exp(filtered_londflow1["alpha_i"]+gamma*filtered_londflow1["log_jobs"]
                                 - beta1*filtered_londflow1["log_distance"])
#or you could do it the easy way like we did last week with the fitted column (See previous practical)
filtered_londflow1.head(10)

,station_origin,station_destination,flows,population,jobs,distance,log_population,log_jobs,log_distance,O_i,D_j,alpha_i,prodsimest1,prodsimest2
0,Abbey Road,Beckton,1,599,442,8510.121774,6.395262,6.091310,9.049012,599,442,3.270351,1.406918,3.329274
1,Abbey Road,Blackwall,3,599,665,3775.448872,6.395262,6.499787,8.236275,599,665,3.270351,3.677350,8.054110
2,Abbey Road,Canary Wharf,1,599,58772,5086.514220,6.395262,10.981421,8.534348,599,58772,3.270351,76.062924,171.386951
3,Abbey Road,Canning Town,37,599,15428,2228.923167,6.395262,9.643939,7.709274,599,15428,3.270351,56.123931,116.908141
4,Abbey Road,Crossharbour,1,599,1208,6686.475560,6.395262,7.096721,8.807842,599,1208,3.270351,3.568414,8.252510
5,Abbey Road,Cutty Sark,2,599,1748,8503.898909,6.395262,7.466228,9.048280,599,1748,3.270351,3.841726,9.090268
6,Abbey Road,Cyprus,7,599,850,6532.099618,6.395262,6.745236,8.784484,599,850,3.270351,2.813752,6.492786
7,Abbey Road,Devons Road,1,599,611,3958.324171,6.395262,6.415097,8.283576,599,611,3.270351,3.326081,7.317636
8,Abbey Road,East India,2,599,1522,3384.141666,6.395262,7.327781,8.126856,599,1522,3.270351,7.359304,15.951298
9,Abbey Road,Island Gardens,2,599,691,7706.296370,6.395262,6.538140,8.949793,599,691,3.270351,2.113923,4.955278


## Assessing the Output

Flow Metrics

In [39]:
#first round the estimates
#filtered_londflow1["prodsimest1"] = round(filtered_londflow1["prodsimest1"],0)
#now we can create a pivot tabel to turn the paired list into a matrix, and compute the margins as well
londflowmat4 = filtered_londflow1.pivot_table(values ="prodsimest2", index="station_origin", columns = "station_destination",
                            aggfunc=np.sum, margins=True)

londflowmat4.fillna(0, inplace=True)

londflowmat4

station_destination,Abbey Road,Acton Central,Acton Town,Aldgate,Aldgate East,All Saints,Alperton,Amersham,Anerley,Angel,...,Wimbledon,Wimbledon Park,Wood Green,Wood Lane,Wood Street,Woodford,Woodgrange Park,Woodside Park,Woolwich Arsenal,All
station_origin,,,,,,,,,,,,,,,,,,,,,
Abbey Road,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,16.885815,1.223169e+03
Acton Central,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.882206e+03
Acton Town,0.000000,0.000000,0.000000,50.819389,51.009187,0.000000,22.062169,0.000000,0.000000,55.134119,...,46.013141,8.917704,15.900445,32.139188,0.000000,0.000000,0.000000,0.000000,0.000000,9.083900e+03
Aldgate,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,57.446710,...,0.000000,0.000000,8.691712,7.033277,0.000000,0.000000,0.000000,0.000000,0.000000,6.147890e+03
Aldgate East,0.000000,0.000000,7.445412,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,57.419503,...,14.734478,0.000000,0.000000,7.273652,0.000000,3.692975,0.000000,3.318350,0.000000,6.876511e+03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Woodford,0.000000,0.000000,22.587860,93.396702,101.437608,0.000000,0.000000,0.000000,0.000000,87.600130,...,46.393716,0.000000,28.433885,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.229352e+04
Woodgrange Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.201332e+03
Woodside Park,0.000000,0.000000,16.353526,55.180444,55.395693,0.000000,0.000000,0.000000,0.000000,65.165190,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.767565e+03


## Goodness of Fit of the Model

In [40]:
def CalcRSquared(observed, estimated):
    """Calculate the r^2 from a series of observed and estimated target values
    inputs:
    Observed: Series of actual observed values
    estimated: Series of predicted values"""

    r, p = scipy.stats.pearsonr(observed, estimated)
    R2 = r **2

    return R2

def CalcRMSE(observed, estimated):
    """Calculate Root Mean Square Error between a series of observed and estimated values
    inputs:
    Observed: Series of actual observed values
    estimated: Series of predicted values"""

    res = (observed -estimated)**2
    RMSE = round(sqrt(res.mean()), 3)

    return RMSE

In [41]:
CalcRSquared(filtered_londflow1["flows"], filtered_londflow1["prodsimest2"])

0.41184805161962407

In [42]:
CalcRMSE(filtered_londflow1["flows"], filtered_londflow1["prodsimest2"])

166.454

## The second value of beta  

In [44]:
# Creating a New variables for beta

alpha_i = prodSim.params[0:398]
gamma = prodSim.params[398]
#beta = -prodSim.params[399]
beta1 = 0.72           # not forgetting the negative in the formular
beta2 = 0.78

In [45]:
alpha_i

station_origin[Abbey Road]          3.270351
station_origin[Acton Central]       5.008886
station_origin[Acton Town]          4.397394
station_origin[Aldgate]             3.361125
station_origin[Aldgate East]        3.408728
                                      ...   
station_origin[Wood Street]         5.672160
station_origin[Woodford]            4.955425
station_origin[Woodgrange Park]     5.320215
station_origin[Woodside Park]       4.496709
station_origin[Woolwich Arsenal]    6.701868
Length: 398, dtype: float64

In [46]:
gamma

0.7301699265793882

In [47]:
beta2

0.78

## Ready to generate the new  estimate with beta = 0.75

In [48]:
filtered_londflow1["prodsimest3"] = np.exp(filtered_londflow1["alpha_i"]+gamma*filtered_londflow1["log_jobs"]
                                 - beta2*filtered_londflow1["log_distance"])
#or you could do it the easy way like we did last week with the fitted column (See previous practical)
filtered_londflow1.head(10)

,station_origin,station_destination,flows,population,jobs,distance,log_population,log_jobs,log_distance,O_i,D_j,alpha_i,prodsimest1,prodsimest2,prodsimest3
0,Abbey Road,Beckton,1,599,442,8510.121774,6.395262,6.091310,9.049012,599,442,3.270351,1.406918,3.329274,1.934432
1,Abbey Road,Blackwall,3,599,665,3775.448872,6.395262,6.499787,8.236275,599,665,3.270351,3.677350,8.054110,4.913596
2,Abbey Road,Canary Wharf,1,599,58772,5086.514220,6.395262,10.981421,8.534348,599,58772,3.270351,76.062924,171.386951,102.705225
3,Abbey Road,Canning Town,37,599,15428,2228.923167,6.395262,9.643939,7.709274,599,15428,3.270351,56.123931,116.908141,73.613768
4,Abbey Road,Crossharbour,1,599,1208,6686.475560,6.395262,7.096721,8.807842,599,1208,3.270351,3.568414,8.252510,4.864903
5,Abbey Road,Cutty Sark,2,599,1748,8503.898909,6.395262,7.466228,9.048280,599,1748,3.270351,3.841726,9.090268,5.282015
6,Abbey Road,Cyprus,7,599,850,6532.099618,6.395262,6.745236,8.784484,599,850,3.270351,2.813752,6.492786,3.832904
7,Abbey Road,Devons Road,1,599,611,3958.324171,6.395262,6.415097,8.283576,599,611,3.270351,3.326081,7.317636,4.451641
8,Abbey Road,East India,2,599,1522,3384.141666,6.395262,7.327781,8.126856,599,1522,3.270351,7.359304,15.951298,9.795556
9,Abbey Road,Island Gardens,2,599,691,7706.296370,6.395262,6.538140,8.949793,599,691,3.270351,2.113923,4.955278,2.896392


## Assessing the Output

Flow Matrics

In [50]:
#first round the estimates
#filtered_londflow1["prodsimest1"] = round(filtered_londflow1["prodsimest1"],0)
#now we can create a pivot tabel to turn the paired list into a matrix, and compute the margins as well
londflowmat5 = filtered_londflow1.pivot_table(values ="prodsimest3", index="station_origin", columns = "station_destination",
                            aggfunc=np.sum, margins=True)

londflowmat5.fillna(0, inplace=True)

londflowmat5

station_destination,Abbey Road,Acton Central,Acton Town,Aldgate,Aldgate East,All Saints,Alperton,Amersham,Anerley,Angel,...,Wimbledon,Wimbledon Park,Wood Green,Wood Lane,Wood Street,Woodford,Woodgrange Park,Woodside Park,Woolwich Arsenal,All
station_origin,,,,,,,,,,,,,,,,,,,,,
Abbey Road,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,9.763263,7.793381e+02
Acton Central,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.678499e+03
Acton Town,0.000000,0.000000,0.000000,28.332729,28.414352,0.000000,13.238861,0.000000,0.00000,30.871697,...,25.772051,5.028368,8.702788,19.005414,0.000000,0.000000,0.000000,0.000000,0.000000,5.194369e+03
Aldgate,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,35.188125,...,0.000000,0.000000,4.931734,3.994977,0.000000,0.000000,0.000000,0.000000,0.000000,3.813449e+03
Aldgate East,0.000000,0.000000,4.147421,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,35.030830,...,8.150244,0.000000,0.000000,4.126696,0.000000,2.079453,0.000000,1.845877,0.000000,4.219209e+03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Woodford,0.000000,0.000000,12.132579,52.290146,57.117838,0.000000,0.000000,0.000000,0.00000,48.663800,...,24.821530,0.000000,15.592835,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.854640e+03
Woodgrange Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.166613e+02
Woodside Park,0.000000,0.000000,8.883873,30.720586,30.814611,0.000000,0.000000,0.000000,0.00000,36.695349,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.346438e+03


## Goodness of fit of the model

In [51]:
def CalcRSquared(observed, estimated):
    """Calculate the r^2 from a series of observed and estimated target values
    inputs:
    Observed: Series of actual observed values
    estimated: Series of predicted values"""

    r, p = scipy.stats.pearsonr(observed, estimated)
    R2 = r **2

    return R2

def CalcRMSE(observed, estimated):
    """Calculate Root Mean Square Error between a series of observed and estimated values
    inputs:
    Observed: Series of actual observed values
    estimated: Series of predicted values"""

    res = (observed -estimated)**2
    RMSE = round(sqrt(res.mean()), 3)

    return RMSE

In [52]:
CalcRSquared(filtered_londflow1["flows"], filtered_londflow1["prodsimest3"])

0.40164019267678236

In [53]:
CalcRMSE(filtered_londflow1["flows"], filtered_londflow1["prodsimest3"])

123.517

## Chceking the three Prodsim estinmates

In [57]:
#check
filtered_londflow1[["prodsimest1", "prodsimest2", "prodsimest3"]]

,prodsimest1,prodsimest2,prodsimest3
0,1.406918,3.329274,1.934432
1,3.677350,8.054110,4.913596
2,76.062924,171.386951,102.705225
3,56.123931,116.908141,73.613768
4,3.568414,8.252510,4.864903
...,...,...,...
43940,131.612217,325.199877,183.874114
43941,268.799374,637.423397,369.872881
43942,36.861579,88.178525,50.886067
43943,106.060284,240.151805,143.469332


from matplotlib import pyplot as plt
_df_8['prodsimest1'].plot(kind='hist', bins=20, title='prodsimest1')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_9['prodsimest2'].plot(kind='hist', bins=20, title='prodsimest2')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_10['prodsimest3'].plot(kind='hist', bins=20, title='prodsimest3')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_11.plot(kind='scatter', x='prodsimest1', y='prodsimest2', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_12.plot(kind='scatter', x='prodsimest2', y='prodsimest3', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_13['prodsimest1'].plot(kind='line', figsize=(8, 4), title='prodsimest1')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_14['prodsimest2'].plot(kind='line', figsize=(8, 4), title='prodsimest2')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_15['prodsimest3'].plot(kind='line', figsize=(8, 4), title='prodsimest3')
plt.gca().spines[['top', 'right']].set_visible(False)

In [56]:
#check
print(filtered_londflow1[["prodsimest1", "prodsimest2", "prodsimest3"]].head(50))

    prodsimest1  prodsimest2  prodsimest3
0      1.406918     3.329274     1.934432
1      3.677350     8.054110     4.913596
2     76.062924   171.386951   102.705225
3     56.123931   116.908141    73.613768
4      3.568414     8.252510     4.864903
5      3.841726     9.090268     5.282015
6      2.813752     6.492786     3.832904
7      3.326081     7.317636     4.451641
8      7.359304    15.951298     9.795556
9      2.113923     4.955278     2.896392
10     2.603262     6.030611     3.551303
11     6.716939    15.054669     9.051881
12     7.361840    16.891581    10.007338
13     5.683201    12.961734     7.708306
14     4.560261    10.185009     6.137511
15     3.591437     7.972610     4.822754
16     2.874532     6.495060     3.885388
17    13.637599    31.492531    18.582317
18    10.208297    23.399123    13.871509
19    11.556615    23.026531    14.911026
20   242.387312   475.030834   310.835323
21    17.352939    32.170457    21.800896
22     9.541767    20.090786    12